In [1]:
%matplotlib inline
import pystan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import scipy
from scipy import stats
import utils
import pickle
from scipy.stats import gaussian_kde
import itertools

### Loading dataset

In [2]:
data = utils.get_data()

### Cross validation

In [3]:
# Unknown and known participants
unknown_part, known_parts = utils.unknown_known()
tr, val = utils.train_val(y=data['y'].to_numpy(), parts=data['participant'].to_numpy(), cond_coher=data['cond_coher'].to_numpy(), cond_spat=data['cond_spat'].to_numpy(), conds=data['conds'].to_numpy())

### Bootstrap

In [4]:
run = "22"
#non parametric bootstrap
rand_parts, boot = utils.nonparaboot(y=tr['tr_y'], parts=tr['tr_parts'], cond_coher=tr['tr_cond_coher'], cond_spat=tr['tr_cond_spat'], conds=tr['tr_conds'], run=run)

# $Model_t$

In [13]:
#set inistial data for molde span
data_winner =  {'N':boot.shape[0],              # Number of trial-level observations
                'ncohers':2,                            # Number of coherence lelvel
                'nspats':2,                             # Number of spatial level
                'nparts':24,                            # Number of participants
                'y':boot['boot_y'],                       # acc*rt in seconds 
                'participant':boot['boot_part'],         # participant index
                'cond_coher':boot['boot_cond_coher'],     # Coherence index
                'cond_spat':boot['boot_cond_spat']};      # sptial index
               
niters = 10000
nwarmup = 2000
nchains = 3
nthin = 2

# initial sampling
initials = utils.init_modelt(y=boot['boot_y'], parts=boot['boot_part'], nchains=nchains)

In [14]:
modelfile = 'stan/model_t.stan'                        #read model span
f = open(modelfile, 'r')
model_wiener = f.read()
# Compile the model
sm = pystan.StanModel(model_code=model_wiener)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_bcb384f635a648399af6a7cb34dc1d3f NOW.


In [15]:
# Train the model and generate samples
fit = sm.sampling(data=data_winner, iter=niters, chains=nchains, warmup=nwarmup, thin=nthin, init = initials)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [16]:
fit


For the full summary use 'print(fit)'

Inference for Stan model: anon_model_bcb384f635a648399af6a7cb34dc1d3f.
3 chains, each with iter=10000; warmup=2000; thin=2; 
post-warmup draws per chain=4000, total post-warmup draws=12000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
deltasd        0.32  8.5e-4   0.06    0.2   0.28   0.32   0.36   0.45   5372    1.0
tersd          0.04  4.6e-5 4.3e-3   0.03   0.04   0.04   0.04   0.05   8602    1.0
alphasd        0.19  4.4e-4   0.04   0.13   0.16   0.19   0.21   0.27   6472    1.0
etasd          0.39  2.6e-3   0.14   0.14   0.28   0.38   0.47   0.69   3019    1.0
deltahier[1]    3.3  1.2e-3    0.1   3.11   3.24    3.3   3.37   3.49   6793    1.0
deltahier[2]   1.99  9.4e-4   0.09   1.83   1.94   1.99   2.05   2.16   8392    1.0
terhier[1]     0.16  8.7e-5 8.0e-3   0.14   0.15   0.16   0.16   0.17   8305    1.0
terhier[2]     0.17  9.0e-5 8.0e-3   0.16   0.17   0.17   0.18   0.19   7926    1.0
alphahier    

### Save model fitting

In [17]:
fit.to_dataframe().to_csv("boot/"+run+"/"+run+"_modelt.csv")

In [18]:
utils.to_pickle(stan_model=sm, stan_fit= fit, save_path="boot/"+run+"/"+run+"_modelt.pkl")

/home/a.ghaderi/Stan/Models/New/model_wihtout_eeg/Original/utils.py:87: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump({"model" : stan_model, "fit" : stan_fit}, f, protocol=pickle.HIGHEST_PROTOCOL)


Saved results to  boot/22/22_modelt.pkl


# $Model_v$

In [21]:
#set inistial data for molde span
data_winner =  {'N':boot.shape[0],                      # Number of trial-level observations
                'nconds':4,                             # Number of conditions
                'nparts':24,                            # Number of participants
                'y':boot['boot_y'],                     # acc*rt in seconds 
                'participant':boot['boot_part'],        # participant index
                'conds':boot['boot_conds']};           # condition index
               
niters = 10000
nwarmup = 2000
nchains = 3
nthin = 2

# initial sampling
initials = utils.init_modelv(y=boot['boot_y'], parts=boot['boot_part'], nchains=nchains)

In [22]:
# Train the model and generate samples
modelfile = 'stan/model_v.stan'                        #read model span
f = open(modelfile, 'r')
model_wiener = f.read()
# Compile the model
sm = pystan.StanModel(model_code=model_wiener)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1e0ae08983f1800825dd1583bbe8bb5c NOW.


In [23]:
# Train the model and generate samples
fit = sm.sampling(data=data_winner, iter=niters, chains=nchains, warmup=nwarmup, thin=nthin, init = initials)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [24]:
fit


For the full summary use 'print(fit)'

Inference for Stan model: anon_model_1e0ae08983f1800825dd1583bbe8bb5c.
3 chains, each with iter=10000; warmup=2000; thin=2; 
post-warmup draws per chain=4000, total post-warmup draws=12000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
deltasd        0.34  2.4e-3   0.06   0.23    0.3   0.34   0.38   0.46    558   1.01
tersd          0.04  8.0e-5 5.9e-3   0.03   0.03   0.04   0.04   0.05   5436    1.0
alphasd        0.21  3.9e-4   0.04   0.15   0.18   0.21   0.23    0.3   9349    1.0
etasd          0.44  8.8e-3   0.15    0.1   0.34   0.43   0.53   0.75    293   1.01
deltahier[1]   3.55  2.8e-3   0.11   3.33   3.47   3.55   3.62   3.77   1660    1.0
deltahier[2]   3.15  1.7e-3   0.11   2.95   3.08   3.15   3.22   3.37   3832    1.0
deltahier[3]   2.12  5.7e-3   0.11   1.92   2.05   2.12   2.19   2.35    343   1.01
deltahier[4]   1.93  1.1e-3   0.09   1.74   1.86   1.93   1.99   2.12   7001    1.0
terhier      

### Save model fitting

In [25]:
fit.to_dataframe().to_csv("boot/"+run+"/"+run+"_modelv.csv")

In [26]:
utils.to_pickle(stan_model=sm, stan_fit= fit, save_path="boot/"+run+"/"+run+"_modelv.pkl")

/home/a.ghaderi/Stan/Models/New/model_wihtout_eeg/Original/utils.py:87: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump({"model" : stan_model, "fit" : stan_fit}, f, protocol=pickle.HIGHEST_PROTOCOL)


Saved results to  boot/22/22_modelv.pkl


# $Model_a$

In [5]:
#set inistial data for molde span
data_winner =  {'N':boot.shape[0],              # Number of trial-level observations
                'ncohers':2,                            # Number of coherence lelvel
                'nspats':2,                             # Number of spatial level
                'nparts':24,                            # Number of participants
                'y':boot['boot_y'],                       # acc*rt in seconds 
                'participant':boot['boot_part'],         # participant index
                'cond_coher':boot['boot_cond_coher'],     # Coherence index
                'cond_spat':boot['boot_cond_spat']};      # sptial index
               
niters = 10000
nwarmup = 2000
nchains = 3
nthin = 2

# initial sampling
initials = utils.init_modela(y=boot['boot_y'], parts=boot['boot_part'], nchains=nchains)

In [6]:
modelfile = 'stan/model_a.stan'                        #read model span
f = open(modelfile, 'r')
model_wiener = f.read()
# Compile the model
sm = pystan.StanModel(model_code=model_wiener)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_9d8a5577c43f0848a0d43c7dc55a8106 NOW.


In [7]:
# Train the model and generate samples
fit = sm.sampling(data=data_winner, iter=niters, chains=nchains, warmup=nwarmup, thin=nthin, init = initials)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [8]:
fit


For the full summary use 'print(fit)'

Inference for Stan model: anon_model_9d8a5577c43f0848a0d43c7dc55a8106.
3 chains, each with iter=10000; warmup=2000; thin=2; 
post-warmup draws per chain=4000, total post-warmup draws=12000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
deltasd        0.32  8.7e-4   0.06   0.21   0.27   0.31   0.36   0.45   4845    1.0
tersd          0.04  6.2e-5 5.9e-3   0.03   0.03   0.04   0.04   0.05   9161    1.0
alphasd         0.2  3.3e-4   0.03   0.15   0.18   0.19   0.21   0.25   6894    1.0
etasd          0.39  2.7e-3   0.14   0.15   0.29   0.38   0.48    0.7   2800    1.0
deltahier[1]   3.31  1.3e-3    0.1   3.12   3.24   3.31   3.37    3.5   5424    1.0
deltahier[2]   2.01  1.1e-3   0.09   1.84   1.95   2.01   2.06   2.18   5699    1.0
terhier        0.16  8.7e-5 7.6e-3   0.15   0.16   0.16   0.17   0.18   7776    1.0
alphahier[1]   1.38  6.0e-4   0.05    1.3   1.35   1.38   1.42   1.48   5828    1.0
alphahier[2] 

### Save model fitting

In [9]:
fit.to_dataframe().to_csv("boot/"+run+"/"+run+"_modela.csv")

In [10]:
utils.to_pickle(stan_model=sm, stan_fit= fit, save_path="boot/"+run+"/"+run+"_modela.pkl")

/home/a.ghaderi/Stan/Models/New/model_wihtout_eeg/Original/utils.py:87: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump({"model" : stan_model, "fit" : stan_fit}, f, protocol=pickle.HIGHEST_PROTOCOL)


Saved results to  boot/22/22_modela.pkl


# $Model_p$

In [11]:
#set inistial data for molde span
data_winner =  {'N':boot.shape[0],              # Number of trial-level observations
                'ncohers':2,                            # Number of coherence lelvel
                'nparts':24,                            # Number of participants
                'y':boot['boot_y'],                     # acc*rt in seconds 
                'participant':boot['boot_part'],        # participant index
                'cond_coher':boot['boot_cond_coher']};  # Coherence index
               
niters = 10000
nwarmup = 2000
nchains = 3
nthin = 2

# initial sampling
initials = utils.init_modelp(y=boot['boot_y'], parts=boot['boot_part'], nchains=nchains)

In [12]:
modelfile = 'stan/model_p.stan'                        #read model span
f = open(modelfile, 'r')
model_wiener = f.read()
# Compile the m odel
sm = pystan.StanModel(model_code=model_wiener)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f30747173589f432a8c066a6f1ef21e5 NOW.


In [13]:
# Train the model and generate samples
fit = sm.sampling(data=data_winner, iter=niters, chains=nchains, warmup=nwarmup, thin=nthin, init = initials)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [14]:
fit


For the full summary use 'print(fit)'

Inference for Stan model: anon_model_f30747173589f432a8c066a6f1ef21e5.
3 chains, each with iter=10000; warmup=2000; thin=2; 
post-warmup draws per chain=4000, total post-warmup draws=12000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
deltasd        0.32  1.1e-3   0.06    0.2   0.28   0.32   0.36   0.46   3545    1.0
tersd          0.04  7.9e-5 6.0e-3   0.03   0.03   0.04   0.04   0.05   5729    1.0
alphasd        0.21  4.2e-4   0.04   0.15   0.18    0.2   0.23   0.29   8047    1.0
etasd           0.4  3.2e-3   0.14   0.15   0.29   0.39   0.49    0.7   1914    1.0
deltahier[1]    3.3  1.4e-3    0.1   3.11   3.24    3.3   3.37   3.49   4545    1.0
deltahier[2]    2.0  1.2e-3   0.09   1.84   1.95    2.0   2.06   2.18   5505    1.0
terhier        0.16  7.9e-5 7.6e-3   0.15   0.16   0.16   0.17   0.18   9132    1.0
alphahier      1.45  5.5e-4   0.05   1.35   1.42   1.45   1.48   1.54   7518    1.0
etahier      

In [15]:
fit.to_dataframe().to_csv("boot/"+run+"/"+run+"_modelp.csv")

In [16]:
utils.to_pickle(stan_model=sm, stan_fit= fit, save_path="boot/"+run+"/"+run+"_modelp.pkl")

/home/a.ghaderi/Stan/Models/New/model_wihtout_eeg/Original/utils.py:87: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump({"model" : stan_model, "fit" : stan_fit}, f, protocol=pickle.HIGHEST_PROTOCOL)


Saved results to  boot/22/22_modelp.pkl


In [17]:
print(1)

1
